In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
!pip install mtcnn

from PIL import Image
from mtcnn import MTCNN
import cv2
from matplotlib import pyplot as plt
import os
from os import listdir
from os.path import isfile, join

In [39]:
input_directory = '/content/drive/MyDrive/face_detection/inputs'
output_directory = '/content/drive/MyDrive/face_detection/outputs'
bounding_boxes_file = join(output_directory, 'bounding_boxes.txt')

files = [f for f in listdir(input_directory) if isfile(join(input_directory, f))]

In [122]:
detector = MTCNN()
output = {}

with open(bounding_boxes_file, 'w') as f:

    for file in files:
        image_path = join(input_directory, file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        faces = detector.detect_faces(image)

        fig, ax = plt.subplots()
        ax.imshow(image)

        f.write(f"{file}:\n")

        new_file = file.replace('.jpg','')
        new_file = '2003/01/01/big/' + new_file
        output[new_file] = []

        for face in faces:
            x, y, width, height = face['box']
            rect = plt.Rectangle((x, y), width, height, fill=False, color='red', linewidth=2)
            ax.add_patch(rect)
            f.write(f"  - ({x}, {y}, {width}, {height})\n")

            if len(output[new_file]) == 0:
              output[new_file] = [[x, y, width, height]]
            else:
              output[new_file].append([x, y, width, height])

        plt.axis('off')
        output_path = join(output_directory, f"processed_{file}")
        plt.savefig(output_path)
        plt.close()
        f.write("\n")

1/1 [==============================] - 0s 30ms/step


In [133]:
print(output)

{'2003/01/01/big/img_326': [[82, 64, 164, 228]], '2003/01/01/big/img_82': [[165, 88, 78, 101]], '2003/01/01/big/img_327': [[75, 96, 166, 228]], '2003/01/01/big/img_158': [[41, 57, 240, 277]], '2003/01/01/big/img_176': [[127, 46, 60, 80]], '2003/01/01/big/img_151': [[134, 96, 55, 70]], '2003/01/01/big/img_230': [[129, 31, 104, 142], [333, 93, 53, 62], [32, 66, 47, 66], [282, 78, 39, 50]], '2003/01/01/big/img_136': [[117, 36, 56, 75]], '2003/01/01/big/img_331': [[143, 43, 100, 122], [47, 100, 88, 122]], '2003/01/01/big/img_346': [[217, 69, 79, 94], [302, 58, 65, 101], [123, 59, 55, 83]], '2003/01/01/big/img_475': [[48, 92, 179, 235]], '2003/01/01/big/img_457': [[179, 40, 89, 126]], '2003/01/01/big/img_458': [[82, 93, 150, 221]], '2003/01/01/big/img_336': [[117, 35, 80, 109], [278, 47, 68, 92]], '2003/01/01/big/img_532': [[274, 121, 80, 106], [108, 27, 81, 109], [200, 81, 50, 70], [52, 110, 47, 57], [359, 105, 53, 79], [3, 141, 22, 29]], '2003/01/01/big/img_496': [[125, 40, 124, 171]], '2

# Test

In [60]:
def filter_coordinate(c, max_value):
    return max(0, min(c, max_value))

def ellipse_to_rectangle(major_axis, minor_axis, angle, center_x, center_y, image_width, image_height):
    """ Converts ellipse parameters to a bounding rectangle. """
    # Calculate angles where the ellipse width and height are maximum
    t = math.atan(-(minor_axis / major_axis) * math.tan(angle))
    t2 = t + math.pi

    # Calculate extreme x points
    x1 = center_x + (major_axis * math.cos(t) * math.cos(angle) - minor_axis * math.sin(t) * math.sin(angle))
    x2 = center_x + (major_axis * math.cos(t2) * math.cos(angle) - minor_axis * math.sin(t2) * math.sin(angle))
    x_min = filter_coordinate(min(x1, x2), image_width)
    x_max = filter_coordinate(max(x1, x2), image_width)

    # Calculate extreme y points
    if math.tan(angle) != 0:
        tan_t = (minor_axis / major_axis) * (1 / math.tan(angle))
    else:
        tan_t = (minor_axis / major_axis) * (1 / (math.tan(angle) + 0.0001))
    t = math.atan(tan_t)
    t2 = t + math.pi

    y1 = center_y + (minor_axis * math.sin(t) * math.cos(angle) + major_axis * math.cos(t) * math.sin(angle))
    y2 = center_y + (minor_axis * math.sin(t2) * math.cos(angle) + major_axis * math.cos(t2) * math.sin(angle))
    y_min = filter_coordinate(min(y1, y2), image_height)
    y_max = filter_coordinate(max(y1, y2), image_height)

    return [int(x_min), int(y_min), int(x_max - x_min), int(y_max - y_min)]

In [61]:
from PIL import Image
def parse_fddb_annotations(fold_dir, images_directories):
    annotations = {}
    fold_files = [f for f in os.listdir(fold_dir) if 'ellipseList.txt' in f]

    for fold_file in fold_files:
        file_path = os.path.join(fold_dir, fold_file)

        with open(file_path, 'r') as file:
            while True:
                line = file.readline().strip()
                if not line:
                    break

                if line in :
                    new_line = line.replace('2003/01/01/big/', '')
                    img_path = os.path.join('/content/drive/MyDrive/face_detection/inputs', new_line + '.jpg')  # Ensure this path is correct
                    image = Image.open(img_path)
                    image_width, image_height = image.size
                    num_faces = int(file.readline().strip())
                    faces = []
                    for _ in range(num_faces):
                        face_info = list(map(float, file.readline().strip().split()))
                        rectangle = ellipse_to_rectangle(
                            major_axis=face_info[0],
                            minor_axis=face_info[1],
                            angle=face_info[2],
                            center_x=face_info[3],
                            center_y=face_info[4],
                            image_width=image_width,
                            image_height=image_height
                        )
                        faces.append(rectangle)
                    annotations[line] = faces

    return annotations

In [89]:
fold_directory = '/content/drive/MyDrive/ScreenInterview/FDDB-folds'
all_annotations = parse_fddb_annotations(fold_directory, images_directories)
print(all_annotations)

{'2003/01/01/big/img_136': [[115, 26, 64, 95]], '2003/01/01/big/img_327': [[64, 46, 179, 277]], '2003/01/01/big/img_151': [[133, 78, 58, 91]], '2003/01/01/big/img_547': [[87, 17, 146, 233]], '2003/01/01/big/img_82': [[163, 75, 84, 119]], '2003/01/01/big/img_588': [[167, 21, 90, 141]], '2003/01/01/big/img_496': [[124, 28, 132, 184]], '2003/01/01/big/img_457': [[175, 16, 98, 151]], '2003/01/01/big/img_336': [[107, 22, 96, 137], [275, 36, 76, 105]], '2003/01/01/big/img_558': [[276, 9, 112, 207]], '2003/01/01/big/img_698': [[175, 10, 72, 102]], '2003/01/01/big/img_458': [[69, 40, 187, 285]], '2003/01/01/big/img_176': [[123, 19, 70, 113]], '2003/01/01/big/img_532': [[108, 6, 86, 138], [195, 67, 60, 91], [266, 97, 94, 137], [358, 91, 51, 91]], '2003/01/01/big/img_475': [[44, 39, 188, 301]], '2003/01/01/big/img_346': [[214, 31, 86, 141], [281, 37, 84, 121]], '2003/01/01/big/img_230': [[31, 45, 52, 92], [118, 15, 112, 162], [276, 69, 47, 69], [332, 83, 55, 75]], '2003/01/01/big/img_158': [[21,

In [ ]:
for nm in all_annotations:
  n = len(all_annotations[nm])
  if nm not in output:
    # print(nm, output.keys())
    output[nm] = []
    for i in range(n):
      output[nm].append([0,0,0,0])
  elif n > len(output[nm]):
    for i in range(n - len(output[nm])):
      output[nm].append([0,0,0,0])

In [127]:
print(output)

{'2003/01/01/big/img_326': [[82, 64, 164, 228]], '2003/01/01/big/img_82': [[165, 88, 78, 101]], '2003/01/01/big/img_327': [[75, 96, 166, 228]], '2003/01/01/big/img_158': [[41, 57, 240, 277]], '2003/01/01/big/img_176': [[127, 46, 60, 80]], '2003/01/01/big/img_151': [[134, 96, 55, 70]], '2003/01/01/big/img_230': [[129, 31, 104, 142], [333, 93, 53, 62], [32, 66, 47, 66], [282, 78, 39, 50]], '2003/01/01/big/img_136': [[117, 36, 56, 75]], '2003/01/01/big/img_331': [[143, 43, 100, 122], [47, 100, 88, 122]], '2003/01/01/big/img_346': [[217, 69, 79, 94], [302, 58, 65, 101], [123, 59, 55, 83]], '2003/01/01/big/img_475': [[48, 92, 179, 235]], '2003/01/01/big/img_457': [[179, 40, 89, 126]], '2003/01/01/big/img_458': [[82, 93, 150, 221]], '2003/01/01/big/img_336': [[117, 35, 80, 109], [278, 47, 68, 92]], '2003/01/01/big/img_532': [[274, 121, 80, 106], [108, 27, 81, 109], [200, 81, 50, 70], [52, 110, 47, 57], [359, 105, 53, 79], [3, 141, 22, 29]], '2003/01/01/big/img_496': [[125, 40, 124, 171]], '2

In [111]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    x1_max = x1 + w1
    y1_max = y1 + h1
    x2_max = x2 + w2
    y2_max = y2 + h2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1_max, x2_max)
    yi2 = min(y1_max, y2_max)

    inter_width = max(xi2 - xi1, 0)
    inter_height = max(yi2 - yi1, 0)
    inter_area = inter_width * inter_height

    box1_area = w1 * h1
    box2_area = w2 * h2

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area if union_area else 0

    return iou

def match_boxes(predicted_boxes, ground_truth_boxes, threshold=0.5):
    matches = []

    matched_gt_indices = set()

    for pred_idx, pred_box in enumerate(predicted_boxes):
        best_match = None
        highest_iou = 0

        for gt_idx, gt_box in enumerate(ground_truth_boxes):
            if gt_idx in matched_gt_indices:
                continue

            iou = calculate_iou(pred_box, gt_box)
            if iou > highest_iou:
                highest_iou = iou
                best_match = gt_idx

        if highest_iou >= threshold:
          matches.append((pred_box, ground_truth_boxes[best_match], highest_iou))
          matched_gt_indices.add(best_match)

        else:
          print(f"No match found for predicted box {pred_box}")

    return matches


In [130]:
from google.colab.patches import cv2_imshow
iou_scores = []
for key in output:
  predicted_boxes = output[key]
  ground_truth_boxes = all_annotations[key]
  new_key = key.replace('2003/01/01/big/', '')
  print(f'For {new_key}:')
  matches_iou = match_boxes(predicted_boxes, ground_truth_boxes)
  for pred_box, gt_box, iou in matches_iou:
    iou_scores.append(iou)
    print(f"Predicted Box: {pred_box} matched with Ground Truth Box: {gt_box} with IoU: {iou:.2f}")

  image_path = '/content/drive/MyDrive/face_detection/inputs/' + new_key + '.jpg'
  image = cv2.imread(image_path)
  for (x, y, w, h) in ground_truth_boxes:
      cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Green
  for (x, y, w, h) in predicted_boxes:
      cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)  # red

  cv2_imshow(image)

  print('\n')

Output hidden; open in https://colab.research.google.com to view.

In [132]:
num_tr_faces = 0
for key, value in all_annotations.items():
  num_tr_faces += len(value)

print(f'Average IoU = ',sum(iou_scores) / num_tr_faces)

Average IoU =  0.687805890880325
